In [ ]:
import os
from collections import defaultdict
from PIL import Image
from shutil import move
import shutil

# Dir for your dataset
src_dir = './datasets'
# For the most, folder most, for the third most, third
dst_dir = './most'

os.makedirs(dst_dir, exist_ok=True)

image_sizes = defaultdict(int)

for filename in os.listdir(src_dir):
    filepath = os.path.join(src_dir, filename)

    if os.path.isfile(filepath) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        try:
            with Image.open(filepath) as img:
                width, height = img.size
                size = (width, height)

                image_sizes[size] += 1
        except PermissionError:
            print(f"Warning: Could not access file {filename}. Skipping.")


# Find max_size
# max_size = max(image_sizes, key=image_sizes.get)

sorted_sizes = sorted(image_sizes.items(), key=lambda x: x[1], reverse=True)

# Get the image size with the third-highest count
third_max_size = sorted_sizes[2][0] if len(sorted_sizes) > 2 else None


for filename in os.listdir(src_dir):
    filepath = os.path.join(src_dir, filename)

    if os.path.isfile(filepath) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        try:
            with Image.open(filepath) as img:
                size = img.size

                if size == third_max_size:
                    move(filepath, os.path.join(dst_dir, filename))
        except PermissionError:
            print(f"Warning: Could not move file {filename}. Skipping.")

print(f'Moved images with size {third_max_size} to {dst_dir}.')


In [ ]:
import cv2

# Need to do it for most and third
path = './most'
cover_path = './train/cover/class1'
hidden_path = './train/hidden/class1'
valid_cover_path = './valid/cover/class1'
valid_hidden_path = './valid/hidden/class1'

image_files = sorted([f for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')])[:422]


valid_files = sorted([f for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')])[422:]

# remove last one
valid_files = valid_files[:-1]

for image_file in image_files:

    # Read the image file
    image = cv2.imread(path + image_file)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Write the grayscale image to directory cover or hidden
    if len(os.listdir(cover_path)) < len(image_files) / 2:
        cv2.imwrite(cover_path + image_file, gray_image)
    else:
        cv2.imwrite(hidden_path + image_file, gray_image)

    # Delete the original image file
    os.remove(path + image_file)

for valid_file in valid_files:

    # Read the image file
    image = cv2.imread(path + valid_file)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Write the grayscale image to directory cover or hidden
    if len(os.listdir(valid_cover_path)) < len(valid_files) / 2:
        cv2.imwrite(valid_cover_path + valid_file, gray_image)
    else:
        cv2.imwrite(valid_hidden_path + valid_file, gray_image)

    # Delete the original image file
    os.remove(path + valid_file)

In [ ]:

image_directory = './most'
encoded_image_directory = 'encoded_datasets'

image_paths = [
        os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.endswith((".jpg", ".jpeg", ".png"))
    ]
image_paths.sort()
image_paths = image_paths[:422]


group1 = image_paths[:len(image_paths)//2]
group2 = image_paths[len(image_paths)//2:]

# create two folders, folder 1 to store cover images, folder 2 to store hidden images
folder1 = os.path.join(image_directory, 'folder1')
folder2 = os.path.join(image_directory, 'folder2')
os.makedirs(folder1, exist_ok=True)
os.makedirs(folder2, exist_ok=True)

for image_path in group1:
    image_filename = os.path.basename(image_path)
    shutil.copy2(image_path, os.path.join(folder1, image_filename))

for image_path in group2:
    image_filename = os.path.basename(image_path)
    shutil.copy2(image_path, os.path.join(folder2, image_filename))
